In [ ]:
from models.network_swinir import *
import os
from collections import OrderedDict
from datetime import datetime
import json
import re
import glob

In [2]:
ckpt_path = "/home/data1/musong/workspace/python/2026-CVPR/KAIR/superresolution/swinir_sr_realworld_x4_psnr/models/20000_G.pth"

In [ ]:
model = SwinIR(upscale=4,
               in_chans=3,
               img_size=64,
               window_size=8,
               img_range=1.0,
               depths=[6, 6, 6, 6, 6, 6],
               embed_dim=180,
               num_heads=[6, 6, 6, 6, 6, 6],
               mlp_ratio=2,
               upsampler='nearest+conv',
               resi_connection='1conv',
               init_type='default',
               scale=4)


ckpt_path = "/home/data1/musong/workspace/python/2026-CVPR/KAIR/superresolution/swinir_sr_realworld_x4_psnr/models/20000_G.pth"
state_dict = torch.load(ckpt_path, map_location='cpu')

model.load_state_dict(state_dict, strict=True)
model.eval()
print("Loaded SwinIR weights from:", ckpt_path)

In [ ]:
device = torch.device("cuda:6")
upscale = 4
window_size = 8
height = (1024 // upscale // window_size + 1) * window_size
width = (720 // upscale // window_size + 1) * window_size

model = model.to(device)

In [ ]:
import torch
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms as T

# path to your image
path = "/home/data1/musong/workspace/python/2026-CVPR/KAIR/trainsets/trainH/0001.png"

# 1. Read image
img = Image.open(path).convert("RGB")  # ensure 3 channels

# 2. Plot it
# plt.imshow(img)
# plt.axis("off")
# plt.show()

# 3. Convert to tensor in [0, 1], shape [1, 3, W, H]
transform = T.ToTensor()  # converts to (C, H, W) and scales to [0,1]
img = transform(img).unsqueeze(0)   # (1, 3, H, W)
resize = T.Resize((height, width))
img = resize(img).to(device)

In [ ]:
plt.imshow(img.squeeze(0).permute(1, 2, 0).cpu().numpy()) 

sr = model(img)

In [ ]:
plt.imshow(sr.squeeze(0).permute(1, 2, 0).detach().cpu().numpy())  # convert to (H, W, C) for plt